In [18]:
from NAS import utils, error_metrics
import pandas as pd
from utils import getDataLaser, getDataMGS, getDataLorenz, getDataDDE
import numpy as np
import math
from utils import getDataSunspots
from reservoirpy.observables import (mse)

In [ ]:



trainX, trainY, valX, valY, testX, testY = getDataSunspots()
nas_architecture = {'nodes': [{'type': 'Input', 'params': {'input_dim': 1}}, {'type': 'Ridge', 'params': {'output_dim': 1, 'ridge': 8.815207271991613e-05}}, {'type': 'NVAR', 'params': {'delay': 3, 'order': 1, 'strides': 3}}, {'type': 'RLS', 'params': {'output_dim': 1, 'alpha': 0.5630569056502455}}, {'type': 'Ridge', 'params': {'output_dim': 1, 'ridge': 3.611959023747037e-05}}], 'edges': [[0, 1], [0, 2], [0, 3], [1, 4], [2, 4], [3, 4]]}

model = utils.constructModel(nas_architecture)


def nrmse(y_true, y_pred):
    mseError = mse(y_true, y_pred)
    variance = np.asarray(y_true).var()
    error = np.sqrt(mseError/variance)
    if math.isnan(error):
        return np.inf
    else:
        return error
    
print(utils.evaluateArchitecture(nas_architecture, np.concatenate([trainX, valX]), np.concatenate([trainY, valY]), testX, testY, [nrmse]))

0.35836079932984


In [ ]:


trainX, trainY, valX, valY, testX, testY = getDataLaser()
architecture = {'nodes': [{'type': 'Input', 'params': {'input_dim': 1}}, {'type': 'Reservoir', 'params': {'units': 1024, 'lr': 0.41, 'sr': 0.906, 'input_connectivity': 0.84, 'rc_connectivity': 0.84}}, {'type': 'Ridge', 'params': {'output_dim': 1, 'ridge': 8.1e-07}}], 'edges': [[0, 1], [1, 2]]}

model = utils.constructModel(architecture)

print(utils.evaluateArchitectureAutoRegressive(architecture, trainX, trainY, valX, valY, [error_metrics.nrmse]))

0.6888956093872487


In [32]:
trainX, trainY, valX, valY, testX, testY = getDataMGS()
architecture = {'nodes': [{'type': 'Input', 'params': {'input_dim': 1}}, {'type': 'Reservoir', 'params': {'units': 2048, 'lr': 0.68, 'sr': 1.406, 'input_connectivity': 0.44, 'rc_connectivity': 0.44}}, {'type': 'Ridge', 'params': {'output_dim': 1, 'ridge': 6.0e-07}}], 'edges': [[0, 1], [1, 2]]}
nas_architecture = {'nodes': [{'type': 'Input', 'params': {'input_dim': 1}}, {'type': 'Reservoir', 'params': {'units': 1660, 'lr': 0.34427701292927426, 'sr': 1.2270381377057633, 'input_connectivity': 0.4147305060237401, 'rc_connectivity': 0.49087298684772573}}, {'type': 'IPReservoir', 'params': {'units': 447, 'lr': 0.5359760595377036, 'sr': 0.7741157049691293, 'mu': 0.011353677276304014, 'sigma': 0.5861691946445886, 'learning_rate': 0.005713532714542338, 'input_connectivity': 0.26747551255227403, 'rc_connectivity': 0.32848065021813627}}, {'type': 'IPReservoir', 'params': {'units': 1160, 'lr': 0.4061401883872777, 'sr': 0.9122281135382351, 'mu': -0.09077876757772718, 'sigma': 1.691081200369488, 'learning_rate': 0.0006859369898355882, 'input_connectivity': 0.21486952140283205, 'rc_connectivity': 0.25929905832981254}}, {'type': 'Ridge', 'params': {'output_dim': 1, 'ridge': 9.55510430296289e-05}}, {'type': 'Ridge', 'params': {'output_dim': 1, 'ridge': 8.335295996470314e-05}}], 'edges': [[0, 1], [0, 2], [1, 3], [1, 4], [2, 5], [3, 5], [4, 5]]}

model = utils.constructModel(nas_architecture)

print(utils.evaluateArchitectureAutoRegressive(nas_architecture, trainX, trainY, valX, valY, [error_metrics.nrmse]))

0.8885270549699857


In [38]:
import numpy as np
import math

trainX, trainY, valX, valY, testX, testY = getDataLorenz()
architecture = {'nodes': [{'type': 'Input', 'params': {'input_dim': 3}}, {'type': 'Reservoir', 'params': {'units': 2048, 'lr': 0.88, 'sr': 0.995, 'input_connectivity': 0, 'rc_connectivity': 0}}, {'type': 'Ridge', 'params': {'output_dim': 3, 'ridge': 1.1e-06}}], 'edges': [[0, 1], [1, 2]]}
nas_architecture = {'nodes': [{'type': 'Input', 'params': {'input_dim': 3}}, {'type': 'NVAR', 'params': {'delay': 4, 'order': 1, 'strides': 2}}, {'type': 'LMS', 'params': {'output_dim': 3, 'alpha': 0.5369376397330149}}, {'type': 'RLS', 'params': {'output_dim': 3, 'alpha': 0.6645404270828781}}, {'type': 'NVAR', 'params': {'delay': 4, 'order': 2, 'strides': 2}}, {'type': 'Ridge', 'params': {'output_dim': 3, 'ridge': 2.7276796630261358e-05}}], 'edges': [[0, 1], [0, 2], [1, 3], [0, 4], [2, 5], [3, 5], [4, 5]]}

model = utils.constructModel(architecture)

print(utils.evaluateArchitectureAutoRegressive(nas_architecture, trainX, trainY, valX, valY, [error_metrics.nrmse]))

0.004954054210418568


In [54]:
from utils import readSavedExperiment
from NAS.error_metrics import r_squared, nrmse
from NAS.utils import runModel

trainX, trainY, valX, valY, testX, testY = getDataMGS()

def findBestGaArchitecture(ga):
    errors = [errors[0] for errors in ga.fitnesses]
    gaBestError = min(errors)
    gaBestErrorIndex = errors.index(gaBestError)
    gaBestModel = ga.architectures[gaBestErrorIndex]
    return gaBestModel

total = 0
gas = [readSavedExperiment(f'./backup_50/mgs/backup_{i}.obj') for i in range(5)]
for ga in gas:
    model = ga.bestModel
    model = utils.trainModel(model, trainX, trainY)
    runModel(model, trainX)
    
    prevOutput = valX[0]
    preds = []
    for _ in range(len(valX)):
        pred = runModel(model, prevOutput)
        prevOutput = pred
        preds.append(pred[0])
    preds = np.array(preds)
    err1 = nrmse(valY, preds)
    err2 = utils.evaluateArchitectureAutoRegressive(findBestGaArchitecture(ga), trainX, trainY, valX, valY, [nrmse])
    print(err1, err2)
    total+=err1

print(total/5)

0.07899601065852414 0.8536957249043801
0.024973521969396218 0.23012103260699987
0.04098128372202721 0.34533367718211744
0.016373293151741967 0.42441931279925205
0.052649667571064235 0.11599376668093746
0.04279475541455075


In [5]:
trainX, trainY, valX, valY, testX, testY = getDataDDE()
architecture = {'nodes': [{'type': 'Input', 'params': {'input_dim': 6}}, {'type': 'Reservoir', 'params': {'units': 256, 'lr': 0.84, 'sr': 0.995, 'input_connectivity': 0.91, 'rc_connectivity': 0.91}}, {'type': 'Ridge', 'params': {'output_dim': 6, 'ridge': 8.3e-07}}], 'edges': [[0, 1], [1, 2]]}

model = utils.constructModel(architecture)

print(utils.evaluateArchitectureAutoRegressive(architecture, trainX, trainY, valX, valY, [error_metrics.nrmse, error_metrics.r_squared]))

229.71790303488274
